In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

23/03/04 18:56:37 WARN Utils: Your hostname, Bronx.local resolves to a loopback address: 127.0.0.1; using 192.168.1.112 instead (on interface en0)
23/03/04 18:56:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/04 18:56:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [7]:
"""
SELECT
    -- Revenue grouping
    date_trunc('hour', lpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
    FROM 
        green
    WHERE
        lpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY 1, 2
"""

"\nSELECT\n    -- Revenue grouping\n    date_trunc('hour', lpep_pickup_datetime) AS hour,\n    PULocationID AS zone,\n    SUM(total_amount) AS amount,\n    COUNT(1) AS number_records\n    FROM \n        green\n    WHERE\n        lpep_pickup_datetime >= '2020-01-01 00:00:00'\n    GROUP BY 1, 2\n"

In [10]:
# df is built on top of rdd of rows
df_green.rdd.take(5)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), lpep_dropoff_datetime=datetime.datetime(2020, 1, 22, 13, 45, 58), store_and_fwd_flag='N', RatecodeID=1, PULocationID=244, DOLocationID=41, passenger_count=1, trip_distance=5.22, fare_amount=22.0, extra=0.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=22.8, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 17, 54, 10), lpep_dropoff_datetime=datetime.datetime(2020, 1, 23, 18, 1, 2), store_and_fwd_flag='N', RatecodeID=1, PULocationID=236, DOLocationID=263, passenger_count=1, trip_distance=0.87, fare_amount=6.5, extra=1.0, mta_tax=0.5, tip_amount=1.2, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=12.25, payment_type=1, trip_type=1, congestion_surcharge=2.75),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 19, 10, 23, 37), lpep_dropo

In [11]:
df_green.take(5)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), lpep_dropoff_datetime=datetime.datetime(2020, 1, 22, 13, 45, 58), store_and_fwd_flag='N', RatecodeID=1, PULocationID=244, DOLocationID=41, passenger_count=1, trip_distance=5.22, fare_amount=22.0, extra=0.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=22.8, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 17, 54, 10), lpep_dropoff_datetime=datetime.datetime(2020, 1, 23, 18, 1, 2), store_and_fwd_flag='N', RatecodeID=1, PULocationID=236, DOLocationID=263, passenger_count=1, trip_distance=0.87, fare_amount=6.5, extra=1.0, mta_tax=0.5, tip_amount=1.2, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=12.25, payment_type=1, trip_type=1, congestion_surcharge=2.75),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 19, 10, 23, 37), lpep_dropo

In [3]:
rdd = df_green \
        .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
        .rdd

In [13]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), PULocationID=244, total_amount=22.8),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 17, 54, 10), PULocationID=236, total_amount=12.25),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 19, 10, 23, 37), PULocationID=166, total_amount=5.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 21, 14, 25, 16), PULocationID=152, total_amount=17.46),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 7, 9, 46), PULocationID=51, total_amount=22.01)]

In [4]:
# instead of lambda we can use python functions

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

# lambda function which can be inside filter:
# lambda row: row.lpep_pickup_datetime >= start

In [5]:
from datetime import datetime
start = datetime(year=2020, month=1, day=1)
rdd.filter(filter_outliers).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), PULocationID=244, total_amount=22.8)]

In [7]:
row = rdd.take(10)[0]

In [9]:
row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)

datetime.datetime(2020, 1, 22, 13, 0)

In [15]:

# key = (Hour, Zone)
# value - (amt, count)

def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)
    
    return (key, value)


In [16]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .take(10)


[((datetime.datetime(2020, 1, 22, 13, 0), 244), (22.8, 1)),
 ((datetime.datetime(2020, 1, 23, 17, 0), 236), (12.25, 1)),
 ((datetime.datetime(2020, 1, 19, 10, 0), 166), (5.3, 1)),
 ((datetime.datetime(2020, 1, 21, 14, 0), 152), (17.46, 1)),
 ((datetime.datetime(2020, 1, 7, 9, 0), 51), (22.01, 1)),
 ((datetime.datetime(2020, 1, 10, 8, 0), 32), (52.68, 1)),
 ((datetime.datetime(2020, 1, 15, 10, 0), 130), (78.36, 1)),
 ((datetime.datetime(2020, 1, 26, 16, 0), 7), (12.09, 1)),
 ((datetime.datetime(2020, 1, 24, 21, 0), 244), (42.0, 1)),
 ((datetime.datetime(2020, 1, 27, 23, 0), 65), (15.3, 1))]

In [17]:
# reduce by key
# input rdd (key, value) ---> output (key, reduced value)


In [21]:
# reduce
def calculate_revenue(left_value, right_value):
   
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
   
    return (output_amount, output_count)

In [26]:
# this is a great example of filter, map, reduce 
# filter is similar to select, if filters the RDD
# map prepares for grouping, we take each row, and transform then into (key, value) 
#     where key is the (hour, zone), since we will group them by hour and by zone
#     where value is (amount, count), since we wanted to calculate sum of amount and count them
# reduce is when we do the final calculation of grouping them and calculating

rdd_new = rdd \
            .filter(filter_outliers) \
            .map(prepare_for_grouping) \
            .reduceByKey(calculate_revenue)



In [27]:
rdd_new.take(10)

Exception in thread "serve RDD 27 with partitions 0" java.net.SocketTimeoutException: Accept timed out
	at java.base/sun.nio.ch.NioSocketImpl.timedAccept(NioSocketImpl.java:694)
	at java.base/sun.nio.ch.NioSocketImpl.accept(NioSocketImpl.java:738)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:690)
	at java.base/java.net.ServerSocket.platformImplAccept(ServerSocket.java:655)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:631)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:588)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:546)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)
Exception in thread "serve RDD 32 with partitions 0" java.net.SocketTimeoutException: Accept timed out
	at java.base/sun.nio.ch.NioSocketImpl.timedAccept(NioSocketImpl.java:694)
	at java.base/sun.nio.ch.NioSocketImpl.accept(NioSocketImpl.java:738)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java

[((datetime.datetime(2020, 1, 21, 13, 0), 33), (535.8900000000001, 26)),
 ((datetime.datetime(2020, 1, 18, 11, 0), 128), (33.51, 1)),
 ((datetime.datetime(2020, 1, 19, 9, 0), 166), (782.1999999999998, 46)),
 ((datetime.datetime(2020, 1, 15, 21, 0), 42), (336.1400000000001, 23)),
 ((datetime.datetime(2020, 1, 4, 12, 0), 116), (119.85, 8)),
 ((datetime.datetime(2020, 1, 8, 20, 0), 197), (10.3, 1)),
 ((datetime.datetime(2020, 1, 19, 0, 0), 25), (142.75000000000003, 11)),
 ((datetime.datetime(2020, 1, 3, 15, 0), 166), (639.56, 39)),
 ((datetime.datetime(2020, 1, 24, 19, 0), 166), (739.6099999999997, 48)),
 ((datetime.datetime(2020, 1, 27, 8, 0), 63), (99.96000000000001, 3))]

In [32]:
from collections import namedtuple
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [33]:
### make it pretty by unnesting and add row names

def unwrap(row):
    return RevenueRow(
            hour=row[0][0], 
            zone=row[0][1], 
            revenue=row[1][0], 
            count=row[1][1])

In [37]:
from pyspark.sql import types
result_schema = types.StructType([
     types.StructField('hour', types.TimestampType(), True), 
     types.StructField('zone', types.IntegerType(), True), 
     types.StructField('revenue', types.DoubleType(), True), 
     types.StructField('count', types.IntegerType(), True)
    ])

In [38]:
df_result = rdd_new.map(unwrap) \
                   .toDF(result_schema)

In [41]:
df_result.write.parquet('tmp/green-revenue')

23/03/04 20:09:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [44]:
# what is map partition, example we want a service that predicts the duration of a trip
# we will define our rdd here
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
                .select(columns) \
                .rdd

In [46]:
# example of map partition
# 

def apply_model_in_batch(partition):
    return [1]

# collect returns a list, 
# this is going to return [1, 1, 1, 1] which means, it's returning 1 for each partition and there are four partition
rdd.mapPartitions(apply_model_in_batch).collect()

[1, 1, 1, 1, 1, 1, 1, 1]

In [48]:
def apply_model_in_batch(partition):
    # partition is not a list, but it's an iterator
    cnt = 0
    for row in partition:
        cnt = cnt + 1
    return [cnt]
rdd.mapPartitions(apply_model_in_batch).collect()

[746744, 418184, 219220, 215980, 212159, 199320, 183795, 109115]

In [49]:
import pandas as pd

In [50]:
rows = duration_rdd.take(10)

In [52]:
def apply_model_in_batch(rows):
    # same results as above but using pands df
    df = pd.DataFrame(rows, columns=columns)
    cnt = len(df)
    return [cnt]

In [53]:
duration_rdd.mapPartitions(apply_model_in_batch).collect()

[746744, 418184, 219220, 215980, 212159, 199320, 183795, 109115]

In [54]:
# let's pretend we have a fancy ML model

def model_predict(df):
    y_pred = df.trip_distance * 5
    return y_pred

In [56]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions
    for row in df.itertuples():
        yield row

In [58]:
duration_rdd.mapPartitions(apply_model_in_batch).take(10)

[Pandas(Index=0, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-22 13:18:32'), PULocationID=244, DOLocationID=41, trip_distance=5.22, predicted_duration=26.099999999999998),
 Pandas(Index=1, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-23 17:54:10'), PULocationID=236, DOLocationID=263, trip_distance=0.87, predicted_duration=4.35),
 Pandas(Index=2, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-19 10:23:37'), PULocationID=166, DOLocationID=166, trip_distance=0.63, predicted_duration=3.15),
 Pandas(Index=3, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-21 14:25:16'), PULocationID=152, DOLocationID=238, trip_distance=2.71, predicted_duration=13.55),
 Pandas(Index=4, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-07 09:46:00'), PULocationID=51, DOLocationID=3, trip_distance=2.13, predicted_duration=10.649999999999999),
 Pandas(Index=5, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-10 08:19:00'), PULocationID=32, DOLocationID=186, trip_distance=

Exception in thread "serve RDD 84" java.net.SocketTimeoutException: Accept timed out
	at java.base/sun.nio.ch.NioSocketImpl.timedAccept(NioSocketImpl.java:694)
	at java.base/sun.nio.ch.NioSocketImpl.accept(NioSocketImpl.java:738)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:690)
	at java.base/java.net.ServerSocket.platformImplAccept(ServerSocket.java:655)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:631)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:588)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:546)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [59]:
duration_rdd.mapPartitions(apply_model_in_batch).toDF().show()

+-----+--------+--------------------+------------+------------+-------------+------------------+
|Index|VendorID|lpep_pickup_datetime|PULocationID|DOLocationID|trip_distance|predicted_duration|
+-----+--------+--------------------+------------+------------+-------------+------------------+
|    0|     2.0|                  {}|         244|          41|         5.22|26.099999999999998|
|    1|     2.0|                  {}|         236|         263|         0.87|              4.35|
|    2|     2.0|                  {}|         166|         166|         0.63|              3.15|
|    3|     2.0|                  {}|         152|         238|         2.71|             13.55|
|    4|     NaN|                  {}|          51|           3|         2.13|10.649999999999999|
|    5|     NaN|                  {}|          32|         186|        17.93|             89.65|
|    6|     2.0|                  {}|         130|         228|        24.04|120.19999999999999|
|    7|     2.0|              

In [60]:
df_predicts = duration_rdd \
              .mapPartitions(apply_model_in_batch) \
              .toDF() \
              .drop('Index')

In [ ]:
df_predicts.select('predicted_duration').show()